## Construindo um GPT

Notebook original por Andrej Karpathy mostrado em [Zero To Hero](https://karpathy.ai/zero-to-hero.html).
Adaptado em português por Lucas Parteka.

In [1]:
# Vamos lê-lo e inspecioná-lo
with open('entrada.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [2]:
print("quantidade de caracteres no dataset: ", len(text))

quantidade de caracteres no dataset:  690812


Em inglês, o dataset tinha 1115394 caracteres, foram usados como dataset trechos de obras do Shakespeare em português.

In [3]:
# Vamos ver os primeiros 10000 caracteres
print(text[:1000])

SANSÃO 
Por minha palavra, Gregório: não devemos levar desaforo para casa. 
 
GREGÓRIO 
É certo; para não ficarmos desaforados. 
 
SANSÃO 
O que quero dizer é que quando eu fico encolerizado puxo logo da 
espada. 
 
GREGÓRIO 
Sim, mas se quiseres viver, toma cuidado para não ficares 
encolarinhado. 
 
SANSÃO 
Quando me irritam, eu ataco prontamente. 
 
GREGÓRIO  
 Mas não te irritas prontamente para atacar. 
 
SANSÃO 
Até um cachorro da casa dos Montecchios me deixa irritado. 
 
GREGÓRIO 
Ficar irritado é pôr-se em movimento, e ser valente é estacar. Logo, 
se ficares irritado, pôr-te-ás a correr. 
 
SANSÃO 
Um cachorro daquela casa me fará fazer pé firme. Encostar-me-ei na 
parede contra qualquer homem ou rapariga da casa de Montecchio. 
 
GREGÓRIO 
Isso prova que não passas de um escravo fraco, porque o mais fraco 
é que se encosta à parede. 
 
SANSÃO 
É certo; é por isso que as mulheres, como vasilhas mais fracas, são 
sempre encostadas à parede. Por isso, afastarei da parede os hom

In [4]:
# aqui os caracteres unicos que ocorrem neste texto
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"&'(),-.:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz©ÀÁÂÃÇÉÊÍÓÔÚàáâãçéêíóôõúü—’“”
95


Detalhe, apareceram mais caracteres que em ingles que eram 65. A língua portuguesa tem acentuação e letras específicas como "Ç"

In [5]:
# cria um mapeamento de caracteres em inteiros
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: pega uma string, retorna uma lista de integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: pega uma lista de inteiros, retorna uma string

print(encode("E aí"))
print(decode(encode("E aí")))

[18, 1, 40, 85]
E aí


In [9]:
# vamos encodificar todo o dataset de texto e salvar num torch.tensor
import torch # Foi usado PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # Os 1000 caracteres vistos acima encodificados

torch.Size([690812]) torch.int64
tensor([32, 14, 27, 32, 70, 28,  1,  0, 29, 54, 57,  1, 52, 48, 53, 47, 40,  1,
        55, 40, 51, 40, 61, 57, 40,  8,  1, 20, 57, 44, 46, 86, 57, 48, 54, 11,
         1, 53, 81, 54,  1, 43, 44, 61, 44, 52, 54, 58,  1, 51, 44, 61, 40, 57,
         1, 43, 44, 58, 40, 45, 54, 57, 54,  1, 55, 40, 57, 40,  1, 42, 40, 58,
        40, 10,  1,  0,  1,  0, 20, 31, 18, 20, 75, 31, 22, 28,  1,  0, 72,  1,
        42, 44, 57, 59, 54, 12,  1, 55, 40, 57, 40,  1, 53, 81, 54,  1, 45, 48,
        42, 40, 57, 52, 54, 58,  1, 43, 44, 58, 40, 45, 54, 57, 40, 43, 54, 58,
        10,  1,  0,  1,  0, 32, 14, 27, 32, 70, 28,  1,  0, 28,  1, 56, 60, 44,
         1, 56, 60, 44, 57, 54,  1, 43, 48, 65, 44, 57,  1, 83,  1, 56, 60, 44,
         1, 56, 60, 40, 53, 43, 54,  1, 44, 60,  1, 45, 48, 42, 54,  1, 44, 53,
        42, 54, 51, 44, 57, 48, 65, 40, 43, 54,  1, 55, 60, 63, 54,  1, 51, 54,
        46, 54,  1, 43, 40,  1,  0, 44, 58, 55, 40, 43, 40, 10,  1,  0,  1,  0,
       

In [10]:
# Vamos dividir os dados entre treino  e teste
n = int(0.9*len(data)) # primeiros 90% serão treino, resto validação
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
train_data[:block_size+1]

tensor([32, 14, 27, 32, 70, 28,  1,  0, 29])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"quando a entrada é {context} o alvo: {target}")

quando a entrada é tensor([32]) o alvo: 14
quando a entrada é tensor([32, 14]) o alvo: 27
quando a entrada é tensor([32, 14, 27]) o alvo: 32
quando a entrada é tensor([32, 14, 27, 32]) o alvo: 70
quando a entrada é tensor([32, 14, 27, 32, 70]) o alvo: 28
quando a entrada é tensor([32, 14, 27, 32, 70, 28]) o alvo: 1
quando a entrada é tensor([32, 14, 27, 32, 70, 28,  1]) o alvo: 0
quando a entrada é tensor([32, 14, 27, 32, 70, 28,  1,  0]) o alvo: 29


In [13]:
torch.manual_seed(1337)
batch_size = 4 # Quantas sequencias independentes serão processadas em paralelo?
block_size = 8 # Qual é o tamanho máximo de contexto para as predições?

def get_batch(split):
    # gera um pequeno lote de dados de entrada x e alvos y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('entradas:')
print(xb.shape)
print(xb)
print('alvos:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # tamanho do lote amostral 
    for t in range(block_size): # dimensão temporal
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"quando a entrada é {context.tolist()} o alvo: {target}")

entradas:
torch.Size([4, 8])
tensor([[ 1, 55, 40, 57, 59, 44,  1, 54],
        [57, 50,  8,  1, 44,  1, 59, 54],
        [ 1, 40,  1, 44, 58, 59, 44,  1],
        [52,  1, 44, 51, 44,  1,  0, 16]])
alvos:
torch.Size([4, 8])
tensor([[55, 40, 57, 59, 44,  1, 54, 60],
        [50,  8,  1, 44,  1, 59, 54, 43],
        [40,  1, 44, 58, 59, 44,  1, 51],
        [ 1, 44, 51, 44,  1,  0, 16, 54]])
----
quando a entrada é [1] o alvo: 55
quando a entrada é [1, 55] o alvo: 40
quando a entrada é [1, 55, 40] o alvo: 57
quando a entrada é [1, 55, 40, 57] o alvo: 59
quando a entrada é [1, 55, 40, 57, 59] o alvo: 44
quando a entrada é [1, 55, 40, 57, 59, 44] o alvo: 1
quando a entrada é [1, 55, 40, 57, 59, 44, 1] o alvo: 54
quando a entrada é [1, 55, 40, 57, 59, 44, 1, 54] o alvo: 60
quando a entrada é [57] o alvo: 50
quando a entrada é [57, 50] o alvo: 8
quando a entrada é [57, 50, 8] o alvo: 1
quando a entrada é [57, 50, 8, 1] o alvo: 44
quando a entrada é [57, 50, 8, 1, 44] o alvo: 1
quando a entra

In [14]:
print(xb) # nossa entrada para o transformador

tensor([[ 1, 55, 40, 57, 59, 44,  1, 54],
        [57, 50,  8,  1, 44,  1, 59, 54],
        [ 1, 40,  1, 44, 58, 59, 44,  1],
        [52,  1, 44, 51, 44,  1,  0, 16]])


In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # cada token diretamente lê fora das logits para o próximo token de uma tabela de pesquisa
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx e os targets são ambos (B,T) tensors de integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx é vetor (B, T)  de indices no contexto atual 
        for _ in range(max_new_tokens):
            # têm as predições
            logits, loss = self(idx)
            # foca apenas no ultimo time step
            logits = logits[:, -1, :] # vira (B, C)
            # aplica softmax para ter probabilidades
            probs = F.softmax(logits, dim=-1) # (B, C)
            # amostra da distribuição
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append índices amostrados para a sequência que está rodando
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 95])
tensor(4.9322, grad_fn=<NllLossBackward0>)

RsIDIQtQ.?GzN ;âBúuÂÊ-v!LtrYZ"Áe&bÓÚI k!ÃtkHÔ’ôôãIdHigÉ(eãf—àT,©wDy©Yz'âõââà"b)
FÁ—Ô?Í—;pÊL&ÃTWBNeYE


In [16]:
# cria um otimizador PyTorch
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [17]:
batch_size = 32
for steps in range(100): # aumenta o número de passos para bons resultados...

    # amostra um lote de dados
    xb, yb = get_batch('train')

    # avalia a perda
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.821751594543457


In [18]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Í©pÔÉ ?àzájÍtDühÀzt&'EÂÀÂOoóÚWPqc h(õÚÇÉYÉalcW-õdq.cSÓiilâZnWÊÔáÉTh:OP”GjÔlt!AÂmÃh’Zq tQÍÚIêwÉ;XSõ;HkÉÇUâAmYYà©Ãtêvõ'üLCêôÍHióànçUzú”jBhÓ©ÍnFmüWÉÂTéchXDHÓ“IdD,À—(LSúDCéfIF
oüQXrjQRVOÔá'ÔNNô AAçmYÇ:kRéL,h(àXalçiÔ!RgóE“ãPóy"AõóV“?u(eQ“IÁÂhXgR)ú”Uç &—wxíMóO—zqêCUàYtTVOçiÍíT,á'-ãÁL
wç’âbWF&Aõá:wqUz?RU;vÀUzÓÃÂmlé.é©Óêq F Ué©Q'rí©VOsuNLgvüÉk!elamÂmN:?WakHÇ-ôÔq-àMÀWQuSBpRfI"V'’-Rl-rwjnçUJtêr
Q.?IS-ÍbibYÀLgr’SI
NVÃupLj?'vz?ÉX©ÁTksRF)ÂWÉOÔHo.ÊÔ! AT
D'rq)
ôÇséÂNôCw ÁTQz,É ÀQbSaiàBk’E-Dúq.!lÃTãhyGD?é s"É:â


## Truque matemático self-attention

In [19]:
# exemplo ilustrando como multiplicações matriciais podem ser usadas para uma "agregação baseada nos pesos (weighted aggregation)"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [20]:
# considere o seguinte exemplo:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, tempo, canais
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [21]:
# queremos x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [22]:
# versão 2: usando multiplicação matricial para uma agregação baseada em pesos
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [23]:
# versão 3: usa Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [24]:
# versão 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, tempo, canais
x = torch.randn(B,T,C)

# vamos ver como performa um self-attention de única cabeça
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [25]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Observação:
-  Attention é um **mecanismo de comunicação**. Podem ser visto como nodos em um grafo direcionado apontando um para o outro e agregando informações com uma soma ponderada de todos nodos que apontam para eles, com pesos dependentes dos dados.
- Não há noção de espaço. Attention simplesmente age sobre um conjunto de vetores. É por isso que precisamos codificar posicionalmente os tokens.
- Cada exemplo entre a dimensão do batch é claramente processado completamente independente e nunca "conversa" com os demais
- Em um "encoder", um bloco de attention apenas deleta a única linha que mascara com `tril`, permitindo todos os tokens se comunicarem. Este bloco aqui é chamado de bloco attention "decoder" pois ele tem mascaramento triangular, e é geralmete usado em configurações autoregressivas como modelagem linguística.
- "self-attention" apenas significa que as chaves e valores são produzidas da mesma fonte que as consultas. Em "cross-attention", as consultas ainda são produzidas do x, porém as chaves e valores vem de alguma outra fonte externa (ex. um módulo encoder)
- "Scaled" attention adicionamente divide `wei` por 1/sqrt(head_size). Isso faz quie quando as entradas Q,K são unidades de variância, wei pode ser unidade de variância também e Softmax permanecerá difuso e não saturará muito. Ilustração abaixo.

In [26]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [27]:
k.var()

tensor(1.0449)

In [28]:
q.var()

tensor(1.0700)

In [29]:
wei.var()

tensor(1.0918)

In [30]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [31]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # fica muito pico, converge para one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [32]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variancia
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 de 100 vetores dimensionais
x = module(x)
x.shape


torch.Size([32, 100])

In [33]:
x[:,0].mean(), x[:,0].std() # mean,std de uma característica entre todas entradas do batch

(tensor(0.1469), tensor(0.8803))

In [34]:
x[0,:].mean(), x[0,:].std() # mean,std of uma única entrada do batch, de suas características

(tensor(-9.5367e-09), tensor(1.0000))

In [35]:
# Exemplo de tradução de francês para português:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> as redes neurais são geniais!<END>



### Código finalizado para referência

In [36]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparametros
batch_size = 16 # quantas sequencias independentes serão processadas em paralelo?
block_size = 32 # qual é o tamanho máximo de contexto para as predições?
max_iters = 15000 #aumentar para mais precisao (vai demorar mais para processar)
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# Vamos lê-lo e inspecioná-lo
with open('entrada.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# aqui os caracteres unicos que ocorrem neste texto
chars = sorted(list(set(text)))
vocab_size = len(chars)

# cria um mapeamento de caracteres em inteiros
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: pega uma string, retorna uma lista de integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: pega uma lista de inteiros, retorna uma string


# Fatias de treino e teste
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# carregamento dos dados
def get_batch(split):
    # gera um pequeno lote de dados de entrada x e alvos y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # computa attention scores ("afinidades")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # performa a agregação ponderada dos valores
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: dimensão de incorporação, n_head: o numero de cabeças desejado
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# modelo bigram super simples
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # cada token diretamente lê as logits para o próximo token de uma tabela de pesquisa
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # camada final de normalização
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx e os targets são ambos (B,T) tensors de integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx é vetor (B, T)  de indices no contexto atual 
        for _ in range(max_new_tokens):
            # corta idx para os ultimos tokens block_size
            idx_cond = idx[:, -block_size:]
            # pega as predições
            logits, loss = self(idx_cond)
            # foca apenas no ultimo time step
            logits = logits[:, -1, :] # becomes (B, C)
            # aplica softmax para ter probabilidades
            probs = F.softmax(logits, dim=-1) # (B, C)
            # amostra da distribuição
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append índices amostrados para a sequência que está rodando
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# imprime o numero de parametros no modelo
print(sum(p.numel() for p in m.parameters())/1e6, 'M parametros')

# cria um otimizador PyTorch
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # De vez em quando, avalia a perda no treino e nos conjuntos de validação
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"iteração {iter}: perda do treino {losses['train']:.4f}, perda da validação {losses['val']:.4f}")

    # amostra um lote de dados
    xb, yb = get_batch('train')

    # avalia a perda
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# gera texto do modelo, 20000 caracteres
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=10000)[0].tolist())) #pode alterar se desejar mais ou menos texto


0.213599 M parametros
iteração 0: perda do treino 4.7213, perda da validação 4.7206
iteração 100: perda do treino 2.7223, perda da validação 2.7430
iteração 200: perda do treino 2.5302, perda da validação 2.5608
iteração 300: perda do treino 2.4519, perda da validação 2.4609
iteração 400: perda do treino 2.3768, perda da validação 2.3939
iteração 500: perda do treino 2.3222, perda da validação 2.3299
iteração 600: perda do treino 2.2792, perda da validação 2.2819
iteração 700: perda do treino 2.2375, perda da validação 2.2374
iteração 800: perda do treino 2.2077, perda da validação 2.2171
iteração 900: perda do treino 2.1537, perda da validação 2.1607
iteração 1000: perda do treino 2.1335, perda da validação 2.1215
iteração 1100: perda do treino 2.0962, perda da validação 2.1093
iteração 1200: perda do treino 2.0895, perda da validação 2.0677
iteração 1300: perda do treino 2.0407, perda da validação 2.0364
iteração 1400: perda do treino 2.0268, perda da validação 2.0111
iteração 1500: 